# Use Function Calling to orchestrate multiple sub-agents 

#### Sample dialog 

1. what's my balance [asks for customer id]
2. cool, when's my next pmt due 
3. refi rates are down to 3.5. dude, should i refi?
4. I'm also applying for a new mortgage. can you remind me if I've got any other docs I'm supposed to get you?

First step is to install the pre-requisites packages

In [1]:
# !pip install --upgrade -q -r requirements.txt
# !pip install --upgrade -q boto3 botocore awscli 

In [1]:
import boto3
import logging
from typing import List, Dict

%load_ext autoreload
%autoreload 2

from knowledge_base import BedrockKnowledgeBase
from agent import AgentsForAmazonBedrock

In [2]:
boto3.__version__

'1.34.117'

In [3]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
agents = AgentsForAmazonBedrock()

In [5]:

region = agents.get_region()
account_id = sts_client.get_caller_identity()["Account"]

suffix = f"{region}-{account_id}"
agent_foundation_models = [
    "anthropic.claude-3-haiku-20240307-v1:0",
    "anthropic.claude-3-sonnet-20240229-v1:0", 
    ]

In [6]:
sub_agent_names = ["existing_mortgage_agent", "mortgage_application_agent"]

In [7]:
from bs4 import BeautifulSoup 
import boto3
import json

bedrock_client = boto3.client('bedrock-runtime')

In [8]:
class MortgageToolsList:
    def __init__(self, session_id: str="9999999", session_state: dict={}):
        self._session_id = session_id
        self._session_state = session_state
        self._existing_mortgage_agent_id = agents.get_agent_id_by_name('existing_mortgage_agent')
        self._mortgage_application_agent_id = agents.get_agent_id_by_name('mortgage_application_agent')

    def invoke_existing_mortgage_agent(self, input_text):
        return agents.invoke(input_text, self._existing_mortgage_agent_id,
                                          session_id=self._session_id, session_state=self._session_state)
    def invoke_mortgage_application_agent(self, input_text):
        return agents.invoke(input_text, self._mortgage_application_agent_id,
                                          session_id=self._session_id, session_state=self._session_state)
    def set_session(self, session_id, session_state: dict={}):
        self._session_id = session_id
        self._session_state = session_state

In [9]:
tools = MortgageToolsList("999")

In [10]:
getattr(tools, "invoke_existing_mortgage_agent")("I'm customer 99. When's my pmt due?")

'Your next mortgage payment for customer 99 is due on 2024-07-01.'

In [11]:
getattr(tools, "invoke_mortgage_application_agent")("What docs do I owe you?")

'The only document you still owe is your employment information. You have already provided the proof of income, proof of assets, and credit information documents.'

In [12]:
from datetime import datetime
today = datetime.today().strftime('%b-%d-%Y')

session_state = {
    "promptSessionAttributes": {
        "customer_ID": "498",
        "today": today
    }
}
session_state 

{'promptSessionAttributes': {'customer_ID': '498', 'today': 'Jun-01-2024'}}

In [13]:
tools.set_session("893", session_state)

In [14]:
getattr(tools, "invoke_existing_mortgage_agent")("what's my balance, and how many years until I'm done w/ this sill mortgage?")

'Your current mortgage balance is $150,000 and you have approximately 6 years remaining on your mortgage.'

In [15]:
supervisor_agent_name = "mortgage_supervisor_agent"
function_defs = agents.get_function_defs(supervisor_agent_name)
function_defs 

{'functions': [{'description': 'you are a mortgage bot. you can retrieve the latest details about an existing mortgage on behalf of customers.',
   'name': 'invoke-existing_mortgage_agent',
   'parameters': {'input_text': {'description': 'The text to be processed by the agent.',
     'required': True,
     'type': 'string'}}},
  {'description': 'you are a bot to create, manage, and complete an application for a new mortgage. you help customers know what documentation they already provided and which ones they still need to provide.',
   'name': 'invoke-mortgage_application_agent',
   'parameters': {'input_text': {'description': 'The text to be processed by the agent.',
     'required': True,
     'type': 'string'}}}]}

In [16]:
def map_function_defs_to_tools(function_defs: Dict) -> Dict:
    tools = []
    for function_def in function_defs:
        params_as_properties = {}
        required_params = []
        for param_name, param_details in function_def["parameters"].items():
            params_as_properties[param_name] = {
                "type": param_details["type"],
                "description": param_details["description"],
            }
            if param_details["required"]:
                required_params.append(param_name)
        tool_def = {
            "toolSpec": {
                "name": function_def["name"].replace("-", "_"),
                "description": function_def["description"],
                'inputSchema': {
                    'json': {
                        'type': 'object',
                            'properties': params_as_properties,
                            'required': required_params
                    }
                }
            }
        }
        tools.append(tool_def)
    return tools

In [17]:
map_function_defs_to_tools(function_defs['functions'])

[{'toolSpec': {'name': 'invoke_existing_mortgage_agent',
   'description': 'you are a mortgage bot. you can retrieve the latest details about an existing mortgage on behalf of customers.',
   'inputSchema': {'json': {'type': 'object',
     'properties': {'input_text': {'type': 'string',
       'description': 'The text to be processed by the agent.'}},
     'required': ['input_text']}}}},
 {'toolSpec': {'name': 'invoke_mortgage_application_agent',
   'description': 'you are a bot to create, manage, and complete an application for a new mortgage. you help customers know what documentation they already provided and which ones they still need to provide.',
   'inputSchema': {'json': {'type': 'object',
     'properties': {'input_text': {'type': 'string',
       'description': 'The text to be processed by the agent.'}},
     'required': ['input_text']}}}}]

In [22]:
toolConfigGen = {'tools': map_function_defs_to_tools(function_defs['functions']),
'toolChoice': {
    'auto': {},
    }
}

In [23]:
#Function for caling the Bedrock Converse API...
def converse_with_tools(messages, system='', model_id=agent_foundation_models[0], 
                        tool_config=toolConfigGen, bedrock_client=bedrock_client):
    response = bedrock_client.converse(
        modelId=model_id,
        system=system,
        messages=messages,
        toolConfig=tool_config
    )
    return response

In [24]:
#Function for orchestrating the conversation flow...
def converse(prompt, system='', bedrock_client=bedrock_client, 
             session_id: str=None, tool_config=toolConfigGen,
             verbose=True, respond_direct_from_tool=False):
    #Add the initial prompt:
    messages = []
    messages.append(
        {
            "role": "user",
            "content": [
                {
                    "text": prompt
                }
            ]
        }
    )

    if verbose:
        print(f"\n{datetime.now().strftime('%H:%M:%S')} - Initial prompt:\n{json.dumps(messages, indent=2)}")

    #Invoke the model the first time:
    output = converse_with_tools(messages, system, tool_config=tool_config)
    if verbose:
        print(f"\n{datetime.now().strftime('%H:%M:%S')} - Output so far:\n{json.dumps(output['output'], indent=2, ensure_ascii=False)}")

    #Add the intermediate output to the prompt:
    messages.append(output['output']['message'])

    function_calling = next((c['toolUse'] for c in output['output']['message']['content'] if 'toolUse' in c), None)

    #Check if function calling is triggered:
    if function_calling:
        #Get the tool name and arguments:
        tool_name = function_calling['name']
        tool_args = function_calling['input'] or {}
        
        #Run the tool:
        if session_id is not None:
            tools.set_session(session_id)
        if verbose:
            print(f"\n{datetime.now().strftime('%H:%M:%S')} - Running ({tool_name}) tool...")
        tool_response = getattr(tools, tool_name)(**tool_args)

        if respond_direct_from_tool: 
            if verbose: 
                print(tool_response)
            return tool_response
        else:
            #Add the tool result to the prompt:
            messages.append(
                {
                    "role": "user",
                    "content": [
                        {
                            'toolResult': {
                                'toolUseId':function_calling['toolUseId'],
                                'content': [
                                    {
                                        "text": tool_response
                                    }
                                ]
                            }
                        }
                    ]
                }
            )
            #print(f"\n{datetime.now().strftime('%H:%M:%S')} - Messages so far:\n{json.dumps(messages, indent=2)}")

            #Invoke the model one more time:
            output = converse_with_tools(messages, system, bedrock_client=bedrock_client)
            if verbose:
                print(f"\n{datetime.now().strftime('%H:%M:%S')} - Final output:\n{json.dumps(output['output'], indent=2, ensure_ascii=False)}\n")
                return output['output']['message']['content'][0]['text']
            else:
                return ""


In [25]:
output = converse(system = [{
    "text": """
You're provided with a set of tools that act as agents on your behalf. 
Don't make reference to the tools in your final answer.
Let the answer from the tool pass through back to the user as is. Do not add any embellishment.
"""}],
        prompt = "I'm customer 99. what's my balance?",
        session_id="33")


16:00:02 - Initial prompt:
[
  {
    "role": "user",
    "content": [
      {
        "text": "I'm customer 99. what's my balance?"
      }
    ]
  }
]



16:00:03 - Output so far:
{
  "message": {
    "role": "assistant",
    "content": [
      {
        "toolUse": {
          "toolUseId": "tooluse_n95ld7ZgT0CQ32xiHiHHOQ",
          "name": "invoke_existing_mortgage_agent",
          "input": {
            "input_text": "customer 99 mortgage balance"
          }
        }
      }
    ]
  }
}

16:00:03 - Running (invoke_existing_mortgage_agent) tool...

16:00:05 - Final output:
{
  "message": {
    "role": "assistant",
    "content": [
      {
        "text": "The outstanding mortgage balance for customer 99 is $150,000."
      }
    ]
  }
}



In [26]:
output

'The outstanding mortgage balance for customer 99 is $150,000.'

In [27]:
output = converse(system = [{
    "text": """
You're provided with a set of tools that act as agents on your behalf. 
Don't make reference to the tools in your final answer.
Let the answer from the tool pass through back to the user as is. Do not add any embellishment.
"""}],
        prompt = "I'm customer 99. what's my balance?",
        session_id="456",
        tool_config=toolConfigGen,
        respond_direct_from_tool=True)


16:00:09 - Initial prompt:
[
  {
    "role": "user",
    "content": [
      {
        "text": "I'm customer 99. what's my balance?"
      }
    ]
  }
]



16:00:11 - Output so far:
{
  "message": {
    "role": "assistant",
    "content": [
      {
        "toolUse": {
          "toolUseId": "tooluse_xAQBK-MUTDick2amsQD60w",
          "name": "invoke_existing_mortgage_agent",
          "input": {
            "input_text": "customer 99 balance"
          }
        }
      }
    ]
  }
}

16:00:11 - Running (invoke_existing_mortgage_agent) tool...
The outstanding principal balance for customer 99 is $150,000.


In [28]:
output 

'The outstanding principal balance for customer 99 is $150,000.'

In [29]:
import uuid 
import time
import numpy as np

def query_loop_by_function_calling(query, num_invokes):
    latencies = []
    for i in range(num_invokes):
        _session_id = str(uuid.uuid1())
        _start_time = time.time()

        output = converse(system = [{
            "text": """
        You're provided with a set of tools that act as agents on your behalf. 
        Don't make reference to the tools in your final answer.
        Let the answer from the tool pass through back to the user as is. Do not add any embellishment.
        """}],
                prompt = query,
                session_id=_session_id,
                tool_config=toolConfigGen,
                verbose=False,
                respond_direct_from_tool=True)

        _end_time = time.time()
        latencies.append(_end_time - _start_time)

    print(f'\n\nInvoked by function calling {num_invokes} times.')
    # get sum of total latencies
    total_time = sum(latencies)
    # get average latency
    avg_time = total_time / num_invokes
    # get p90 latency
    p90_time = np.percentile(latencies, 90)

    print(f'Average latency: {avg_time:.1f}, P90 latency: {p90_time:.1f}')

In [30]:
query_loop_by_function_calling("I am customer 999. how many years until the mortgage maturity date?", 
                               25)



Invoked by function calling 25 times.
Average latency: 3.8, P90 latency: 4.2
